In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


Arquivo = "MICRODADOS_ENEM_2020_1-300.csv"
DF= pd.read_csv(Arquivo, engine="python", encoding='ISO-8859-1')
DF2 = pd.DataFrame([])
def minmax(df):
    mi = df.min()
    ma = df.max()
    sai = df.apply(lambda x: (x-mi)/(ma-mi))
    return sai

In [2]:
%%time
DF2["IDADE"] = minmax(DF["TP_FAIXA_ETARIA"])

CPU times: total: 0 ns
Wall time: 13.6 ms


# Sexo

M	Masculino
F	Feminino 

Categorizado como:
masculino = 1
Feminino = 0

MASCULINO

In [3]:
%%time
DF2['MASCULINO'] = DF['TP_SEXO'].apply(lambda x: 0 if x=='M' else 1)


CPU times: total: 0 ns
Wall time: 7.42 ms


# Estado civil

0	Não informado

1	Solteiro(a)

2	Casado(a)/Mora com companheiro(a)

3	Divorciado(a)/Desquitado(a)/Separado(a)

4	Viúvo(a)

Criado 3 novas categorias:
SOLTEIRO: 1


In [4]:
DF2["SOLTEIRO"] = DF["TP_ESTADO_CIVIL"].apply(lambda x: 1 if x==1 else 0)


# Cor Raça

0	Não declarado

1	Branca

2	Preta

3	Parda

4	Amarela

5	Indígena 


Criado 5 novas categorias

BRANCA: 1

PRETA: 2




In [5]:
DF2["BRANCA"] = DF["TP_COR_RACA"].apply(lambda x: 1 if x==1 else 0)
DF2["PRETA"] = DF["TP_COR_RACA"].apply(lambda x: 1 if (x==2 or x==3) else 0)


# Situação de Conclusão

TP_ST_CONCLUSAO

1	Já concluí o Ensino Médio

2	Estou cursando e concluirei o Ensino Médio em 2022

3	Estou cursando e concluirei o Ensino Médio após 2022

4	Não concluí e não estou cursando o Ensino Médio

Resumido a uma categoria:

EM CONCLUIDO:1

EM NO ANO DA AVALIACAO: 2

EM APOS O ANO DA AVALIACAO: 3


In [6]:
DF["TP_ST_CONCLUSAO"].describe()

count    19286.000000
mean         1.460023
std          0.607988
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          4.000000
Name: TP_ST_CONCLUSAO, dtype: float64

In [7]:
DF2["EM CONCLUIDO"] = DF["TP_ST_CONCLUSAO"].apply(lambda x: 1 if x==1 else 0)
DF2["EM NO ANO DA AVALIACAO"] = DF["TP_ST_CONCLUSAO"].apply(lambda x: 1 if x==2 else 0)
DF2["EM APOS O ANO DA AVALIACAO"] = DF["TP_ST_CONCLUSAO"].apply(lambda x: 1 if x==3 else 0)

# Tipo da escola

TP_ESCOLA

1	Não Respondeu

2	Pública

3	Privada

Uma categoria:
PUBLICA: 1


In [8]:
m = DF["TP_ESCOLA"].size
DF["TP_ESCOLA"].value_counts()/m


1    0.652650
2    0.336876
3    0.010474
Name: TP_ESCOLA, dtype: float64

In [9]:
DF2["Escola Pública"] = DF["TP_ESCOLA"].apply(lambda x: 1 if x==1 else 0)

# treineiro

In [10]:
DF2["Treineiro"] = DF["IN_TREINEIRO"].apply(lambda x: 1 if x==1 else 0)

# Q001
## Até que série seu pai, ou o homem responsável por você, estudou?

# Escolaridade do Pai

A	Nunca estudou.

B	Não completou a 4ª série/5º ano do Ensino Fundamental.

C	Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.

D	Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.

E	Completou o Ensino Médio, mas não completou a Faculdade.

F	Completou a Faculdade, mas não completou a Pós-graduação.

G	Completou a Pós-graduação.

H	Não sei.

# agrupamento

Nunca estudou: A e H

Completou no máximo o ensino médio:B, C, D e E

Completou Faculdade ou pós graduação: F e G


In [11]:
DF2["Escolaridade Pai A"] = DF["Q001"].apply(lambda x: 1 if (x=="A") else 0)
DF2["Escolaridade Pai B"] = DF["Q001"].apply(lambda x: 1 if (x=="B" or x=="C" or x=="D"or x=="E") else 0)
DF2["Escolaridade Pai C"] = DF["Q001"].apply(lambda x: 1 if (x=="F" or x=="G") else 0)


# Q002
## Até que série sua mãe, ou a mulher responsável por você, estudou?

Análogo ao Q001

In [12]:
DF2["Escolaridade Mae A"] = DF["Q002"].apply(lambda x: 1 if (x=="A" or x=="H") else 0)
DF2["Escolaridade Mae B"] = DF["Q002"].apply(lambda x: 1 if (x=="B" or x=="C" or x=="D"or x=="E") else 0)
DF2["Escolaridade Mae C"] = DF["Q002"].apply(lambda x: 1 if (x=="F" or x=="G") else 0)


# Q003

## A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação do seu pai ou do homem responsável por você. (Se ele não estiver trabalhando, escolha uma ocupação pensando no último trabalho dele).

A	Grupo 1: Lavrador, agricultor sem empregados, bóia fria, criador de animais (gado, porcos, galinhas, ovelhas, cavalos etc.), apicultor, pescador, lenhador, seringueiro, extrativista.

B	Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.

C	Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.

D	Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.

E	Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.

F	Não sei.


## Agrupamento

A - Ocupação com menor requisito de formação: A, B e C
B - Ocupação com maior requisito de formação: D e E

In [13]:
DF2["Ocupação Pai A"] = DF["Q003"].apply(lambda x: 1 if (x=="A" or x=="B" or x=="C") else 0)
DF2["Ocupação Pai B"] = DF["Q003"].apply(lambda x: 1 if (x=="D" or x=="E") else 0)


# Q004

Análogo à Q003

In [14]:
DF2["Ocupação Mae A"] = DF["Q004"].apply(lambda x: 1 if (x=="A" or x=="B" or x=="C") else 0)
DF2["Ocupação Mae B"] = DF["Q004"].apply(lambda x: 1 if (x=="D" or x=="E") else 0)


# Q005

## Incluindo você, quantas pessoas moram atualmente em sua residência?

Quantitativa ordinal de 1 a 20 normalizada entre 0 e 1 por min max.





In [15]:
DF2["pessoas na residencia"] = minmax(DF["Q005"])

# 006

### Agrupamento


Renda
A - Até 1 salário mínimo
B - de 1 a 2,5 salários mínimos
C - de 2,5 até 7 salários mínimo
D - de 7 a 12 salários mínimos
E - acima de 12 salários mínimos

In [16]:
DF2["Renda A"] = DF["Q006"].apply(lambda x: 1 if (x=="A" or x=="B") else 0)
DF2["Renda B"] = DF["Q006"].apply(lambda x: 1 if (x=="C" or x=="D" or x=="E") else 0)
DF2["Renda C"] = DF["Q006"].apply(lambda x: 1 if (x=="F" or x=="G" or x=="H" or x=="I" or x=="J") else 0)
DF2["Renda D"] = DF["Q006"].apply(lambda x: 1 if (x=="K" or x=="L" or x=="M" or x=="N") else 0)
DF2["Renda E"] = DF["Q006"].apply(lambda x: 1 if (x=="O" or x=="P" or x=="Q") else 0)

# Q0024
Residencia tem computador




In [17]:
DF2["Tem computador em casa"] = DF["Q006"].apply(lambda x: 1 if (x=="A") else 0)

# Desempenho


In [18]:
# Cria dicionário para referência de variáveis endógenas

regid     = {'cut_MT':  0,
            'cut_LC':  1,
            'cut_CH':  2,
            'cut_CN':  3,
            'cut_RE':  4,
            'cut_OB':  5,
            'cut_GE':  6,
            'qut_MT':  7,
            'qut_LC':  8,
            'qut_CH':  9,
            'qut_CN':  10,
            'qut_RE':  11,
            'qut_OB':  12,
            'qut_GE':  13}

endogenas = ['cut_MT', 'cut_LC','cut_CH','cut_CN','cut_RE','cut_OB','cut_GE',
            'qut_MT', 'qut_LC','qut_CH','qut_CN','qut_RE','qut_OB','qut_GE'
            ]

In [19]:

DF2["cut_MT"] = pd.cut( DF["NU_NOTA_MT"], bins=[0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_LC"] = pd.cut( DF["NU_NOTA_LC"], bins=[0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_CH"] = pd.cut( DF["NU_NOTA_CH"], bins = [0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_CN"] = pd.cut( DF["NU_NOTA_CN"], bins = [0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_RE"] = pd.cut( DF["NU_NOTA_REDACAO"], bins = [0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_OB"] = pd.cut( (DF["NU_NOTA_CN"]+DF["NU_NOTA_LC"]+DF["NU_NOTA_CH"]+DF["NU_NOTA_MT"])/4, bins = [0, 200, 400,600, 800, 1000], labels= False)
DF2["cut_GE"] = pd.cut( (DF["NU_NOTA_CN"]+DF["NU_NOTA_LC"]+DF["NU_NOTA_CH"]+DF["NU_NOTA_MT"]+DF["NU_NOTA_REDACAO"])/5, bins = [0, 200, 400,600, 800, 1000], labels= False)




In [20]:
S = pd.qcut( (DF["NU_NOTA_CN"]+DF["NU_NOTA_LC"]+DF["NU_NOTA_CH"]+DF["NU_NOTA_MT"]+DF["NU_NOTA_REDACAO"])/5, q=5).value_counts().sort_index()
qins = [0, S.index[1].left, S.index[2].left, S.index[3].left,S.index[4].left, 1000]

In [21]:
print(S)
qins

(160.559, 462.74]     2428
(462.74, 511.2]       2427
(511.2, 558.648]      2423
(558.648, 621.812]    2426
(621.812, 836.14]     2426
dtype: int64


[0, 462.74, 511.2, 558.648, 621.812, 1000]

In [22]:


DF2["qut_MT"] = pd.cut( DF["NU_NOTA_MT"], bins = qins, labels= False)
DF2["qut_LC"] = pd.cut( DF["NU_NOTA_LC"], bins = qins, labels= False)
DF2["qut_CH"] = pd.cut( DF["NU_NOTA_CH"], bins = qins, labels= False)
DF2["qut_CN"] = pd.cut( DF["NU_NOTA_CN"], bins = qins, labels= False)
DF2["qut_RE"] = pd.cut( DF["NU_NOTA_REDACAO"], bins = qins, labels= False)
DF2["qut_OB"] = pd.cut( (DF["NU_NOTA_CN"]+DF["NU_NOTA_LC"]+DF["NU_NOTA_CH"]+DF["NU_NOTA_MT"])/4, bins = qins, labels= False)
DF2["qut_GE"] = pd.cut( (DF["NU_NOTA_CN"]+DF["NU_NOTA_LC"]+DF["NU_NOTA_CH"]+DF["NU_NOTA_MT"]+DF["NU_NOTA_REDACAO"])/5, bins = qins, labels= False)





# Salva DF2

In [23]:
DF2.to_csv(Arquivo.split(".")[0]+"_DF2_.csv")

In [24]:
DF2 = DF2.dropna()

 # Cria vetor com nome das features

In [25]:
features = DF2.drop(endogenas, axis=1).columns

 # Separa em dados de treino e teste

In [26]:
# Divide os dados em conjuntos de treinamento e teste
X_train, X_test, Y_train, Y_test = train_test_split(DF2.drop(endogenas,axis=1), DF2[endogenas], test_size=0.1)


# Regressão Logística




In [27]:

log_reg = LogisticRegression(max_iter = 1000)

regcut_GE = log_reg.fit(X_train,Y_train['cut_GE'])
print(regressao.score(X_test,Y_test['cut_GE']))

regressao = log_reg.fit(X_train,Y_train['cut_GE'])
print(regressao.score(X_test,Y_test['cut_GE']))

0.7290552584670231


In [28]:
from joblib import dump

export = [X_test, Y_test, regressao]

dump(export,"Regressao_geral003_"+Arquivo.split('.')[0]+".txt")

['Regressao_geral003_MICRODADOS_ENEM_2020_1-300.txt']